In [1]:
%load_ext sql
%sql duckdb:///../../database/data.duckdb
%config SqlMagic.displaylimit = 10

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

Connecting to 'duckdb:///../../database/data.duckdb'

# communes

In [6]:
%%sql --save communes
WITH
communes_year AS (
    SELECT
        de_partition AS annee,
        categorie,
        cdparametresiseeaux,
        inseecommune,
        sum(1) AS nb_analyses,
        sum(CASE
            WHEN valtraduite = 0 OR valtraduite = 1 OR valtraduite IS NULL
                THEN 1
            ELSE 0
        END) AS nb_analyses_not_quantify,
        sum(CASE
            WHEN
                limitequal_float IS NOT NULL AND valtraduite > limitequal_float
                THEN 1
            ELSE 0
        END) AS nb_analyses_not_ok,
        sum(CASE
            WHEN
                limitequal_float IS NOT NULL AND valtraduite <= limitequal_float
                THEN 1
            ELSE 0
        END) AS nb_analyses_ok
    FROM
        int__resultats_udi_communes
    WHERE
        categorie = 'cvm'
    GROUP BY
        annee,
        categorie,
        cdparametresiseeaux,
        inseecommune
),

    communes_annee AS (
SELECT
    communes_year.annee,
    communes_year.inseecommune,
    communes_year.categorie,
    communes_year.cdparametresiseeaux,
    'bilan annuel' AS periode,
    nb_analyses,
    nb_analyses_not_quantify,
   nb_analyses_not_ok,
   nb_analyses_ok,
    CASE
        WHEN communes_year.nb_analyses = 0
            THEN 'Pas recherché'
        WHEN
            communes_year.nb_analyses_not_quantify > 0
            AND communes_year.nb_analyses_ok = 0
            THEN 'non quantifié'
        WHEN
            communes_year.nb_analyses > 0
            AND communes_year.nb_analyses_not_ok >= 1
            THEN '> 0,5 µg/L'
        WHEN communes_year.nb_analyses > 0 AND communes_year.nb_analyses_ok >= 1
            THEN '<= 0,5 µg/L'
        ELSE 'Check SQL'
    END AS resultat
FROM
    communes_year),

last_pvl AS (
    SELECT
        inseecommune,
        categorie,
        cdparametresiseeaux,
        datetimeprel,
        valtraduite,
        limitequal_float,
        ROW_NUMBER()
            OVER (
                PARTITION BY cdreseau, cdparametresiseeaux
                ORDER BY datetimeprel DESC
            )
            AS row_number
    FROM
        int__resultats_udi_communes
    WHERE
        categorie = 'cvm' -- à supprimer pour avoir pour tout
),

communes_last AS (
SELECT
    last_pvl.inseecommune,
    last_pvl.categorie,
    last_pvl.cdparametresiseeaux,
    last_pvl.datetimeprel AS last_datetimeprel,
    'dernier prélévement' AS periode,
    CASE
        WHEN
            last_pvl.valtraduite = 0
            OR last_pvl.valtraduite = 1
            OR last_pvl.valtraduite IS NULL
            OR last_pvl.limitequal_float IS NULL
            THEN 'non quantifié'
        WHEN
            last_pvl.limitequal_float IS NOT NULL
            AND last_pvl.valtraduite > last_pvl.limitequal_float
            THEN '> 0,5 µg/L'
        WHEN
            last_pvl.limitequal_float IS NOT NULL
            AND last_pvl.valtraduite <= last_pvl.limitequal_float
            THEN '<= 0,5 µg/L'
        ELSE 'Check SQL'
    END AS resultat
FROM
    last_pvl
WHERE
    last_pvl.row_number = 1
),

last AS (
    SELECT
        inseecommune AS commune_code_insee,
        categorie,
        last_datetimeprel,
        resultat,
        periode
    FROM
        communes_last
),

yearly AS (
    SELECT
        inseecommune AS commune_code_insee,
        categorie,
        NULL AS last_datetimeprel,
        resultat,
        CONCAT(periode, ' - ', annee) AS periode
    FROM
        communes_annee
)


SELECT
    periode,
    commune_code_insee,
    categorie,
    last_datetimeprel,
    resultat
FROM
    last
UNION ALL
SELECT
    periode,
    commune_code_insee,
    categorie,
    last_datetimeprel,
    resultat
FROM
    yearly



Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

periode,commune_code_insee,categorie,last_datetimeprel,resultat
dernier prélévement,82185,cvm,2024-06-20 13:47:00,non quantifié
dernier prélévement,83125,cvm,2024-06-10 08:40:00,non quantifié
dernier prélévement,83065,cvm,2024-12-31 10:53:00,non quantifié
dernier prélévement,83069,cvm,2023-08-31 09:14:00,"<= 0,5 µg/L"
dernier prélévement,83142,cvm,2024-08-12 12:04:00,"<= 0,5 µg/L"
dernier prélévement,83142,cvm,2024-03-11 11:56:00,non quantifié
dernier prélévement,83081,cvm,2024-12-19 09:05:00,non quantifié
dernier prélévement,83081,cvm,2024-09-24 09:01:00,non quantifié
dernier prélévement,83027,cvm,2024-09-03 10:18:00,"<= 0,5 µg/L"
dernier prélévement,83012,cvm,2024-09-03 09:08:00,non quantifié


In [7]:
%%sql --with communes
SELECT DISTINCT
    resultat
FROM
    communes

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

resultat
"> 0,5 µg/L"
non quantifié
"<= 0,5 µg/L"
